# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
DATASET = 'demo_coding_vs_intergenomic_seqs'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [2]:
# Parameters
DATASET = "demo_human_or_worm"


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

demo_human_or_worm 0 64 10


# Data download

In [4]:
from pathlib import Path
import tensorflow as tf
import tensorflow_addons as tfa

import numpy as np
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import get_basic_cnn_model_v0 as get_model

if not is_downloaded(DATASET):
    download_dataset(DATASET)

/home/jovyan/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
2022-06-02 23:27:52.269779: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-02 23:27:52.643500: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43672 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:a3:00.0, compute capability: 8.6
/home/jovyan/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No versio

/home/jovyan/.genomic_benchmarks/fasta/Caenorhabditis_elegans.WBcel235.dna.toplevel.fa.gz: 30.3MB [00:07, 4.03MB/s]                            


Reference /home/jovyan/.genomic_benchmarks/fasta/Homo_sapiens.GRCh38.dna.toplevel.fa.gz already exists. Skipping.


7it [00:00,  8.08it/s]
100%|██████████| 24/24 [00:26<00:00,  1.12s/it]


In [5]:
info(DATASET)

/home/jovyan/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Dataset `demo_human_or_worm` has 2 classes: human, worm.

All lengths of genomic intervals equals 200.

Totally 100000 sequences have been found, 75000 for training and 25000 for testing.


,train,test
human,37500,12500
worm,37500,12500


## TF Dataset object

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]
NUM_CLASSES = len(CLASSES)

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 75000 files belonging to 2 classes.


In [7]:
if NUM_CLASSES > 2:
    train_dset = train_dset.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))

## Text vectorization

In [8]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
VOCAB_SIZE = len(vectorize_layer.get_vocabulary())
vectorize_layer.get_vocabulary()

['', '[UNK]', 'a', 't', 'c', 'g', 'n']

In [9]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [10]:
model = get_model(NUM_CLASSES, VOCAB_SIZE)

In [11]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10


2022-06-02 23:31:58.237866: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204
2022-06-02 23:31:59.687334: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1172/1172 [==============================] - 23s 16ms/step - loss: 0.2635 - binary_accuracy: 0.8954 - f1_score: 0.8663
Epoch 2/10
1172/1172 [==============================] - 19s 16ms/step - loss: 0.1753 - binary_accuracy: 0.9314 - f1_score: 0.9254
Epoch 3/10
1172/1172 [==============================] - 19s 16ms/step - loss: 0.1607 - binary_accuracy: 0.9370 - f1_score: 0.9324
Epoch 4/10
1172/1172 [==============================] - 19s 16ms/step - loss: 0.1527 - binary_accuracy: 0.9404 - f1_score: 0.9359
Epoch 5/10
1172/1172 [==============================] - 19s 16ms/step - loss: 0.1472 - binary_accuracy: 0.9425 - f1_score: 0.9391
Epoch 6/10
1172/1172 [==============================] - 18s 15ms/step - loss: 0.1439 - binary_accuracy: 0.9444 - f1_score: 0.9397
Epoch 7/10
1172/1172 [==============================] - 18s 15ms/step - loss: 0.1410 - binary_accuracy: 0.9450 - f1_score: 0.9409
Epoch 8/10
1172/1172 [==============================] - 19s 16ms/step - loss: 0.1366 - binary_accurac

## Evaluation on the test set

In [12]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

if NUM_CLASSES > 2:
    test_dset = test_dset.map(lambda x, y: (x, tf.one_hot(y, depth=NUM_CLASSES)))
test_ds =  test_dset.map(vectorize_text)

Found 25000 files belonging to 2 classes.


In [13]:
model.evaluate(test_ds)

391/391 [==============================] - 6s 15ms/step - loss: 0.1483 - binary_accuracy: 0.9418 - f1_score: 0.9324


[0.148258239030838, 0.9418399930000305, 0.9324167370796204]